In [9]:
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import seaborn as sns
import math
import cv2 as cv
import imageio
from scipy import spatial
from tqdm import tqdm
import torch

In [10]:
def read_quat(fi='cffk-orientation-dc8bb42/data/c48n9.quat'):
    pdb=open(fi,'r')
    lines=pdb.readlines()
    quats=[]
    for line in lines[4:]:
        quat=np.asarray([float(line[0:12]),
                         float(line[13:25]),
                         float(line[26:38]),
                         float(line[39:51])])
        quats.append(quat)
    return quats
def rotpos(pos,quat=[1,0,0,0],origin=torch.tensor([0,0,0])):   
    pos_rot=torch.mm(pos-origin,quat2mat(quat))
    pos_rot=pos_rot+origin
    return pos_rot
def proj(pos,weight,drift,resolution,size,epsilon=1):
    pic_i=torch.zeros(size,size)
    atom_x=torch.tensor((pos[:,0]+drift[0])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
    atom_y=torch.tensor((pos[:,1]+drift[1])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
    for i in range(pos.shape[0]):
        pic_i[atom_y[i],atom_x[i]]+=weight[i]
    return (pic_i+epsilon).log()
def genlib(pdblib='GAPRPDB',outdir='',
           quatfile='lib/c48u27.quat',edge=1,resolution=3.7,iscg=0,libmol='lib/mol.csv',size=0,level=50):
    rot=read_quat(quatfile)
    lib={'pdb':[],'q':[],'pic':[]}
    datalist=[]
    maxdist=0
    for npdb in os.listdir(pdblib):
        pdb=readpdb(pdblib+'/'+npdb,iscg=iscg,libfile=libmol)
        pos=torch.tensor(np.asarray(pdb)[:,4:7].astype('float32'))
        origin=torch.tensor([(pdb.x*pdb.weight).sum()/pdb.weight.sum(),
                             (pdb.y*pdb.weight).sum()/pdb.weight.sum(),
                             (pdb.z*pdb.weight).sum()/pdb.weight.sum()])
        pos=pos-origin
        maxdist=max((pos*pos).sum(axis=1).max().sqrt(),maxdist)
        weight=torch.tensor(np.asarray(pdb)[:,7].astype('float32'))
        datalist.append((pos,weight,npdb))
    if size==0:
        size=1+int(2*maxdist/resolution)
    drift_ini=torch.tensor([size*resolution/2,size*resolution/2,size*resolution/2])
    for data in datalist:
        pos=data[0]
        weight=data[1]
        npdb=data[2]
        for i in tqdm(range(len(rot))):
#        for i in range(len(rot)):
            pos_rot=rotpos(pos,rot[i])
            pic=proj(pos_rot,weight,drift_ini,resolution,size)
#            ori=[int((size[0]-mat.shape[0])/2),int((size[1]-mat.shape[1])/2)]
            lib['pdb'].append(npdb)
            lib['q'].append(rot[i])
            lib['pic'].append(pic.numpy())
    lib=pd.DataFrame(lib)
    if outdir!='':
        os.mkdir(outdir)
        for i in lib.index:
#            print(lib.pic[i].max())
            img=np.zeros(size,size,3)
            cv.imwrite(outdir+'/'+str(i)+'.png',(lib.pic[i]*level).astype('uint8'))
            outpic=cv.applyColorMap(cv.imread(outdir+'/'+str(i)+'.png'),cv.COLORMAP_AUTUMN)
            cv.imwrite(outdir+'/'+str(i)+'.png',outpic)
    return lib,size

In [11]:
def gen_Morph_parameter(lib,kernalsize_filter=(5,5),thresh=2.5,appsize=20,cviter=1):
    thresh_area=[999,0]
    thresh_dist=[10,0]
    circle=0
    liblist=[]
    for i in lib.index:
#        pic=lib.pic[i]+lib.pic[i].min()
#        projpic=pic*255/(pic.max())
        bi_gray=np.zeros(lib.pic[i].shape)
        bi_gray[lib.pic[i]>thresh]=255
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        largest=0
#        print(i)
        for c, contour in enumerate(contours):
            Area = cv.contourArea(contour)
#            print(c,Area)
            if Area>=largest:
                largest=Area
#                circle=cv.arcLength(contour, True)
                contour_r=contour.reshape(contour.size//2,2)
                maxdist=spatial.distance_matrix(contour_r, contour_r).max()
#        liblist.append([largest,circle])
        liblist.append([largest,maxdist])
#        print(largest,circle*circle/(largest*4*3.1415926))
#        maxcd=circle*circle/(largest*4*3.1415926)
    liblist=np.asarray(liblist)
    thresh_contour = liblist[spatial.ConvexHull(liblist).vertices]
    return thresh_contour
def Extract_Protein(
    picdir,thresh_contour,
    outdir='test',
    cviter=3,
    picave=3,
    diiter=3,
    thresh_adt=0.35,
    thresh_pick=0.1,
    kernalsize_adt=(25,25),
    kernalsize_filter=(5,5),
    avekernal=[0.25,0.5,0.25],
    listthresh=30
    ):
    os.mkdir(outdir+'/binary')
    os.mkdir(outdir+'/ori')
#################################################################################
    extended_contour=np.zeros((thresh_contour.shape[0]+1,thresh_contour.shape[1]))
    extended_contour[:-1,:]=thresh_contour
    avekernal=np.asarray(avekernal)
    avecenter=avekernal.argmax()
    dirlist=os.listdir(picdir)
    pinserie=[]
    contourlist=[]
    print(thresh_contour)
    hullarea=spatial.ConvexHull(thresh_contour).volume
    for i,p in enumerate(dirlist):
        pinframe=[i+avecenter,dirlist[i+avecenter]]
        if i>=(len(dirlist)-picave):
            break
        ave=0
        ori=cv.imread(picdir+'/'+dirlist[i+avecenter],flags=cv.CV_8U)
        for j in range(picave):
            ave=ave+cv.imread(picdir+'/'+dirlist[i+j],flags=cv.CV_8U)*avekernal[j]
        burr=cv.GaussianBlur(ave.astype('uint8'), kernalsize_filter, 0)
        bi_gray= (255-AdaptiveThresh(burr,kernalsize_adt,thresh_adt)).astype('uint8')
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        cv.imwrite(outdir+'/binary/'+dirlist[i+avecenter],bi_gray)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        outpic=ave.copy()
#        tempmask=np.zeros(ave.shape,dtype='uint8')
        pcounts=0
        for c, contour in enumerate(contours):
            # print(i)
#            print(contour)
            Area = cv.contourArea(contour)
            if Area==0:
                continue
#            circle=cv.arcLength(contour, True)
            contour_r=contour.reshape(contour.size//2,2)
            maxdist=spatial.distance_matrix(contour_r, contour_r).max()
            extended_contour[-1]=np.asarray([Area,maxdist])
            para_pick=spatial.ConvexHull(extended_contour).volume-hullarea
            if Area>=50:
                contourlist.append([p,c,Area,maxdist,contour.mean(axis=0)[0][0],contour.mean(axis=0)[0][1],para_pick])
            if para_pick<thresh_pick:
                cv.drawContours(ori,contours,c,255,1)
                pinframe.append([contour.mean(axis=0)[0],contour])
                pcounts+=1
        pinserie.append(pinframe)
#        print('found ',pcounts,' in pic ',dirlist[i+avecenter])    
#        tempmask=cv.dilate(tempmask,kernel=kernel,iterations=3)
#        masked=(tempmask/255*(255-cv.imread(picdir+'/'+dirlist[i+avecenter],flags=cv.CV_8U))).astype('uint8')
        cv.imwrite(outdir+'/ori/'+dirlist[i+avecenter],ori)
    pd.DataFrame(contourlist,columns=['name','id','area','circle','x','y','para_dist']).to_csv(outdir+'/contour.csv')
    return pinserie
def Particle_clustering(pinserie,minobserve,maxmissing=15,maxdrift=7.0):
    plist=[]
    protein=[]
#pinserie saved as [[frame,filename,[center1,contour1],[center2,contour2]...]...]
#clustered protein saved as [[frame1,filename1,center1,contour1], [frame2,filename2,center2,contour2], [frame3,filename3,center3,contour3]...] 
    while len(pinserie)>0:
#        print(len(pinserie),pinserie[0],len(pinserie[0]))
        while len(pinserie[0])==2:
            pinserie.pop(0)
            if len(pinserie)==0:
                if len(protein)>=minobserve:
                    print('protein ',len(plist),'detected in ',len(protein),' frames')
                    plist.append(protein)
#                print('all protein detected')
                return plist
        protein=[[pinserie[0][0],pinserie[0][1],pinserie[0][2][0],pinserie[0][2][1]]]
        print('protein',len(plist),' detected at frame ',protein[-1][0],' centered at ',protein[-1][2])
        pinserie[0].pop(2)
        missedframe=1
        for pl in pinserie[1:]:
#            print(pl[0],len(pl))
            for pi in range(2,len(pl)):
#protein drift less than sqrt(maxdrift*missingframes) been considered as series
                if ((protein[-1][2]-pl[pi][0])**2).sum() <missedframe*maxdrift*maxdrift:
#                    print('find protein',len(plist),' at frame ',pl[0],'centered at',pl[pi][0],'after ',missedframe,' frames')
                    protein.append([pl[0],pl[1],pl[pi][0],pl[pi][1]])
                    pl.pop(pi)
                    missedframe=0
                    break
            missedframe+=1
            if missedframe>=maxmissing:
                break
        if len(protein)>=minobserve:
            print('protein ',len(plist),'detected in ',len(protein),' frames')
            plist.append(protein)
    return plist
def Gen_picked_pic(plist,dirin,outdir,
                   kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)),dilate_iter=3,picsize=[35,35],
                   debug=True):
    count=0
    dirlist=[]
    drifts=[]
    for i in plist:
        count+=1
        os.mkdir(outdir+'/protein_'+str(count))
        os.mkdir(outdir+'/ori_'+str(count))
#        print(os.system('mkdir '+tempdir+'_'+str(count)),'mkdir '+tempdir+'_'+str(count))
        dirlist.append(outdir+'/protein_'+str(count))
        drift=[]
        for c in i:
#            print(c[1],c[3])
            ori_pic=cv.imread(dirin+'/'+c[1],flags=cv.CV_8U)
            mask=np.zeros(ori_pic.shape,dtype='uint8')
            cv.drawContours(mask,(c[3],),0,255,cv.FILLED)
            mask=cv.dilate(mask,kernel=kernel,iterations=dilate_iter)
            masked=(mask/255*(255-ori_pic))
            masked[masked==0]=255-ori_pic.mean()
     #       masked=masked.astype('uint8')
            pos=[int(c[2][0]),int(c[2][1])]
            cutted=masked[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cutted_ori=ori_pic[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cv.drawContours(ori_pic,(c[3],),0,255,1)
            drift.append((max(0,pos[1]-picsize[1]),max(0,pos[0]-picsize[0])))
            cv.imwrite(outdir+'/ori_'+str(count)+'/'+c[1],cutted_ori)
            cv.imwrite(outdir+'/protein_'+str(count)+'/'+c[1],cutted)
        drifts.append(drift)
    return dirlist,drifts
def cvmatch(lib,pic,drift,ori=0,r=10,method=cv.TM_CCOEFF_NORMED):
#def cvmatch(lib,pic,ori=0,r=10,method=cv.TM_CCORR_NORMED):
#def cvmatch(lib,pic,ori=0,r=10,method=cv.TM_CCORR):

#    tempic=(mpimg.imread(pic).mean()-mpimg.imread(pic)).astype('uint8')
#    inpic=cv.imread(pic,cv.CV_8U)[323:366,385:424].astype('float32')
#    inpic=cv.imread(pic,cv.CV_8U)[270:372,425:517].astype('float32')
    tempic=cv.imread(pic,cv.CV_8U).astype('float32')
#    tempic=extract_protein(inpic)
    bestfit=[0,0,0,0,0,0,0]
    cvlib=lib
    scores=[]
    drifts=[]
#    for p in lib:
#        cvlib[p]=(lib[p].astype('float32'))
    for p in cvlib.index:
        i=cvlib.pic[p]
        cmatch=cv.matchTemplate(tempic,i,method)
        m=cmatch.argmax()
        l=len(cmatch[0])
# normalized TMCCORR
#        score=2*cmatch.max()/((i*i).sum()+(tempic[int(m//l):int(m//l)+i.shape[0],m%l:m%l+i.shape[1]]*tempic[int(m//l):int(m//l)+i.shape[0],m%l:m%l+i.shape[1]]).sum())
        score=cmatch.max()
        scores.append(score)
        drifts.append([m%l+i.shape[1],int(m//l)+i.shape[0]])
    return drifts,np.asarray(scores)
def extract_best(scores,libshape,ntops=5):
    score_t=scores.reshape(libshape[1],libshape[0])
    output=[]
    for pid,score_l in enumerate(score_t):
        top_index=np.argpartition(score_l, -ntops)[-ntops:]
        for i in top_index:
            output.append([pid,i,score_l[i]])
    return(np.asarray(output))
def autosearch(picdir,template,
               outdir='fitted',debug=True,
               quatfile='lib/c48u27.quat',resolution=3.7,rf=10,pick=[],
               libmol='lib/mol.csv',iscg=0,picsize=0,thresh_pick=0.01,
               minobserve=100,thresh_adt=0.35,ntops=1):
    dirlist=os.listdir(picdir)
    os.mkdir(outdir)
    if type(template)==str:
        lib,size=genlib(template,quatfile,resolution=resolution,iscg=iscg,libmol=libmol,size=picsize)
        picsize=[size,size]
        print('lib generated')
        thresh_contour=gen_Morph_parameter(lib)
    elif type(template)==pd.DataFrame:
        lib=template
        print('use input lib')
        picsize=lib.pic[0].shape
        thresh_contour=gen_Morph_parameter(lib)
    else:
        raise ValueError('input lib must be a folder contains pdb files or a libary generated by genlib')
    if len(pick)==3:
#            thresh_contour=np.asarray([[pick[0]*pick[1]*np.pi/(4*resolution),max(pick[0],pick[1])],
#                                       [pick[2]*pick[1]*np.pi/(4*resolution),max(pick[2],pick[1])],
#                                       [pick[0]*pick[2]*np.pi/(4*resolution),max(pick[0],pick[2])],])/resolution
            thresh_contour=np.asarray([[pick[0]*pick[1]/(resolution),np.sqrt(pick[0]*pick[0]+pick[1]*pick[1])],
                                       [pick[2]*pick[1]/(resolution),np.sqrt(pick[2]*pick[2]+pick[1]*pick[1])],
                                       [pick[0]*pick[2]/(resolution),np.sqrt(pick[0]*pick[0]+pick[2]*pick[2])],])/resolution
            print('pick particles with length: ',template)
    print('pick range generated')
    pinserie=Extract_Protein(picdir,thresh_contour,outdir=outdir,thresh_adt=thresh_adt,thresh_pick=thresh_pick)
    print('contours found')    
    plist=Particle_clustering(pinserie,minobserve,maxmissing=30,maxdrift=7)
    print('particle clustered')
    picked_dirlist,drifts=Gen_picked_pic(plist,picdir,outdir=outdir,
                                         kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3)),dilate_iter=3,picsize=picsize)
    print('picked picture generated')
    summary=pd.DataFrame({'pic':[],'protein_id':[],'pdb':[],'score':[],'qr':[],'qi':[],'qj':[],'qk':[],'dx':[],'dy':[]},index=[])
    pdblist=lib['pdb'].unique()
    libshape=[int(len(lib)/len(pdblist)),len(pdblist)]
    qlist=lib['q'][0:libshape[0]]
    for pc,protein in enumerate(picked_dirlist):
        fits={}
        dirlist_pro=os.listdir(protein)
        list_score=np.zeros([len(dirlist_pro),len(lib)])
        for c,i in enumerate(dirlist_pro):
            pic_summary=pd.DataFrame({'pic':[],'protein_id':[],'pdb':[],'score':[],'qr':[],'qi':[],'qj':[],'qk':[],'dx':[],'dy':[]},index=[])
            drift,list_score[c]=cvmatch(lib,protein+'/'+i,drifts[pc][c])
            outdata=extract_best(list_score[c],libshape,ntops)
            for out in outdata:
#               print(qlist[out[1]])
#                for n in range(len(out)):
#                print(out)
                pic_summary=pic_summary.append(pd.DataFrame({'pic':c,'protein_id':pc,'pdb':pdblist[int(out[0])],'score':out[2],
                                                     'qr':qlist[out[1]][0].item(),'qi':qlist[out[1]][1].item(),'qj':qlist[out[1]][2].item(),'qk':qlist[out[1]][3].item(),
                                                     'dx':drift[int(out[0]*libshape[1]+out[1])][1],'dy':drift[int(out[0]*libshape[1]+out[1])][0]},index=[0]),ignore_index=True)
#                print(outdata)
            best_index=pic_summary.score.argmax()
#            print(pic_summary.dx[best_index])
            fit=[0,pic_summary.dx[best_index],pic_summary.dy[best_index],pic_summary.score[best_index],
                 pic_summary.pdb[best_index]]
            summary=summary.append(pic_summary,ignore_index=True)
            fits[i]=fit
            print('protein '+protein+' in picture',i,'is fitted as',fit[4],'fitted score:',fit[3])
        makegif(fits,outdir+'/ori_'+str(pc+1),outdir+'/protein_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'pics',picsize)
        print('gif for protein '+str(pc+1)+' saved as '+outdir+'/protein_'+str(pc+1)+'movie.gif')
        table_score=pd.DataFrame(list_score)
        table_score.index=dirlist_pro
        table_score.to_csv(outdir+'/protein_'+str(pc+1)+'.csv')
#        makegif(0,outdir+'/ori_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'ori.gif')
    summary.to_csv(outdir+'/summary.csv')
    return summary

In [12]:
############ module I structure reading ##############
### currently support: all-atom/sirah2 pdb libary
def readpdb(fi='target.pdb',libfile='lib/mol.csv',iscg=False):
    pdb=open(fi,'r')
    index=[]
    name=[]
    resname=[]
    chain=[]
    resid=[]
    x=[]
    y=[]
    z=[]
    weight=[]
    element=[]
    lines=pdb.readlines()
    weightdict={}
    lib=pd.read_csv(libfile)
    if iscg:
        for i in lib.index:
            weightdict[(lib.residue[i],lib.element[i])]=lib.electron[i]
    else:
        for i in lib.index:
            weightdict[lib.element[i]]=lib.electron[i]
    for line in lines:
        if line[0:4] == 'ATOM':
            index.append(int(line[6:11]))
            n=line[12:16]
            name.append(n.strip())
            resname.append(line[17:20].strip())
            resid.append(int(line[22:26]))
            x.append(float(line[30:38]))
            y.append(float(line[38:46]))
            z.append(float(line[46:54]))
            if len(line)<77 or line[76:78].strip()=='':
                element.append(n.strip()[0])
            else:
                element.append(line[76:78].strip())
            if iscg:
                weight.append(weightdict[(resname[-1],name[-1])])
            else:
                weight.append(weightdict[element[-1]])
#            charge.append(int(line[78:80]))
    data=pd.DataFrame({
        'series':index,
        'name':name,
        'resname':resname,
        'resid':resid,
        'x':x,
        'y':y,
        'z':z,
        'weight':weight,
        'element':element
    })
    return data

In [13]:
############ module 0 math ##############
#Tait–Bryan angles https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
#eular=np.asarray[phi,theta,psi]
def eular2mat(eular):
    se=np.sin(eular)
    ce=np.cos(eular)
    mat=np.asarray([[ ce[2]*ce[1]   ,-se[2]*ce[1]+ce[2]*se[1]*se[0]  , se[2]*se[0]+ce[2]*se[1]*ce[0]] ,
                    [ se[2]*ce[1]   , ce[2]*ce[1]+se[2]*se[1]*se[0]  ,-ce[2]*se[0]+se[2]*se[1]*ce[0]] ,
                    [       se[1]   ,                   ce[1]*se[0]  ,                   ce[1]*ce[0]] ])
    return mat
def read_quat(fi='cffk-orientation-dc8bb42/data/c48n9.quat'):
    pdb=open(fi,'r')
    lines=pdb.readlines()
    quats=[]
    for line in lines[4:]:
        quat=np.asarray([float(line[0:12]),
                         float(line[13:25]),
                         float(line[26:38]),
                         float(line[39:51])])
        quats.append(quat)
    return quats
def gen_quater(libgrid='std60.csv',ang=5):#obsoleted
    grids=pd.read_csv(libgrid)
    theta=math.pi/ang
    dtheta=math.pi/ang
    quaterlist=[np.array([1,0,0,0])]
    for i in range(0,ang):
        s=math.sin(theta/2)
        c=math.cos(theta/2)
        for j in grids.index:
            quaterlist.append(np.array([c,s*grids.i[j],s*grids.j[j],s*grids.k[j]]))
        theta+=dtheta
#    print(len(quaterlist),' quaterions generated')
    return quaterlist
def gen_quater_d(libgrid='std60.csv',ang=5):
    grids=pd.read_csv(libgrid)
    quaterlist=[np.array([1,0,0,0])]
    s=math.sin(ang*np.pi/360)
    c=math.cos(ang*np.pi/360)
    for j in grids.index:
        quaterlist.append(np.array([c,s*grids.i[j],s*grids.j[j],s*grids.k[j]]))
#    print(len(quaterlist),' quaterions generated')
    return quaterlist
def eular2quater(eular):
    se=np.sin(eular/2)
    ce=np.cos(eular/2)
    q=np.asarray([ce[0]*ce[1]*ce[2]+se[0]*se[1]*se[2],
                  se[0]*ce[1]*ce[2]-ce[0]*se[1]*se[2],
                  ce[0]*se[1]*ce[2]+se[0]*ce[1]*se[2],
                  ce[0]*ce[1]*se[2]-se[0]*se[1]*ce[2]])
    return q
def quater2eular(q):
    eular=np.asarray([np.arctan2(2*(q[0]*q[1]+q[2]*q[3],1-2*(q[1]*q[1]+q[2]*q[2]))),
                      np.arcsin(2*(q[0]*q[2]-q[1]*q[3])),
                      np.arctan2(2*(q[0]*q[3]+q[1]*q[2],1-2*(q[3]*q[3]+q[2]*q[2])))])
    return eular
def pearson(pica,picb):
    pica_mean=torch.mean(pica)
    picb_mean=torch.mean(picb)
    cov=torch.mean(pica*picb)-pica_mean*picb_mean
    pica_var=torch.mean(pica*pica)-pica_mean*pica_mean
    picb_var=torch.mean(picb*picb)-picb_mean*picb_mean
    loss=cov*cov/(pica_var*picb_var)
    pearson=loss.sqrt()
    return pearson
def rotpos(pos,quat=[1,0,0,0],origin=torch.tensor([0,0,0])):   
    pos_rot=torch.mm(pos-origin,quat2mat(quat))
    pos_rot=pos_rot+origin
    return pos_rot
def quat2mat(q):
#q=(qr,qi,qj,qk)
#from qr=cos(theta/2) qi,qj,qk=sin(theta/2)*x,y,z 
    mat=torch.tensor([[1-2*q[2]*q[2]-2*q[3]*q[3],  2*(q[1]*q[2]-q[3]*q[0]),  2*(q[1]*q[3]+q[2]*q[0])],
                      [  2*(q[1]*q[2]+q[3]*q[0]),1-2*q[1]*q[1]-2*q[3]*q[3],  2*(q[2]*q[3]-q[1]*q[0])],
                      [  2*(q[1]*q[3]-q[2]*q[0]),  2*(q[2]*q[3]+q[1]*q[0]),1-2*q[1]*q[1]-2*q[2]*q[2]]],dtype=torch.float64)
    return mat

In [14]:
############ module II generate libaray ##############
'''def pdb_trans(pdb,trans):
#transform define as (psi,theta,phi,dx,dy)
    trans_pdb=pdb.copy(deep=True)
    pos=np.matmul(np.array(data)[:,4:7],eular2mat(trans[:3]))
    pos[]
    trans_pdb[:,4:7]
    return trans_pdb'''
def proj_sigmoid(pos,weight,drift,resolution,size,N=100,epsilon=1):
    pic_i=torch.zeros(size,size)
    atom_x=torch.tensor((pos[:,0]+drift[0])*N)
    atom_y=torch.tensor((pos[:,1]+drift[1])*N)
    d=resolution*N
    for y in range(size):
        for x in range(size):
            pic_i[y,x]=((((atom_x-d*x).sigmoid()-(atom_x-d*(x+1)).sigmoid())*((atom_y-d*y).sigmoid()-(atom_y-d*(y+1)).sigmoid())*weight).sum()+epsilon).log()
    return pic_i
def proj(pos,weight,drift,resolution,size,epsilon=1):
    pic_i=torch.zeros(size,size)
    atom_x=torch.tensor((pos[:,0]+drift[0])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
    atom_y=torch.tensor((pos[:,1]+drift[1])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
    for i in range(pos.shape[0]):
        pic_i[atom_y[i],atom_x[i]]+=weight[i]
    return (pic_i+epsilon).log()
def genlib(pdblib='GAPRPDB',outdir='',
           quatfile='lib/c48u27.quat',edge=1,resolution=3.7,iscg=0,libmol='lib/mol.csv',size=0,level=50):
    rot=read_quat(quatfile)
    lib={'pdb':[],'q':[],'pic':[]}
    datalist=[]
    maxdist=0
    for npdb in os.listdir(pdblib):
        pdb=readpdb(pdblib+'/'+npdb,iscg=iscg,libfile=libmol)
        pos=torch.tensor(np.asarray(pdb)[:,4:7].astype('float32'))
        origin=torch.tensor([(pdb.x*pdb.weight).sum()/pdb.weight.sum(),
                             (pdb.y*pdb.weight).sum()/pdb.weight.sum(),
                             (pdb.z*pdb.weight).sum()/pdb.weight.sum()])
        pos=pos-origin
        maxdist=max((pos*pos).sum(axis=1).max().sqrt(),maxdist)
        weight=torch.tensor(np.asarray(pdb)[:,7].astype('float32'))
        datalist.append((pos,weight,npdb))
    if size==0:
        size=1+int(2*maxdist/resolution)
    drift_ini=torch.tensor([size*resolution/2,size*resolution/2,size*resolution/2])
    for data in datalist:
        pos=data[0]
        weight=data[1]
        npdb=data[2]
        for i in tqdm(range(len(rot))):
#        for i in range(len(rot)):
            pos_rot=rotpos(pos,rot[i])
            pic=proj(pos_rot,weight,drift_ini,resolution,size)
#            ori=[int((size[0]-mat.shape[0])/2),int((size[1]-mat.shape[1])/2)]
            lib['pdb'].append(npdb)
            lib['q'].append(rot[i])
            lib['pic'].append(pic.numpy())
    lib=pd.DataFrame(lib)
    if outdir!='':
        os.mkdir(outdir)
        for i in lib.index:
#            print(lib.pic[i].max())
            img=np.zeros(size,size,3)
            cv.imwrite(outdir+'/'+str(i)+'.png',(lib.pic[i]*level).astype('uint8'))
            outpic=cv.applyColorMap(cv.imread(outdir+'/'+str(i)+'.png'),cv.COLORMAP_AUTUMN)
            cv.imwrite(outdir+'/'+str(i)+'.png',outpic)
    return lib,size

In [15]:
############ module I&III morphing ##############
def AdaptiveThresh(I,winSize,ratio=0.15):
    I_mean=cv.boxFilter(I,cv.CV_8U,winSize,borderType=cv.BORDER_REFLECT)
    out=I-(1.0-ratio)*I_mean
    out[out>=0]=255
    out[out<0]=0
    return out.astype(np.uint8)
def gen_Morph_parameter(lib,kernalsize_filter=(5,5),thresh=2.5,appsize=20,cviter=1):
    thresh_area=[999,0]
    thresh_dist=[10,0]
    circle=0
    liblist=[]
    for i in lib.index:
#        pic=lib.pic[i]+lib.pic[i].min()
#        projpic=pic*255/(pic.max())
        bi_gray=np.zeros(lib.pic[i].shape)
        bi_gray[lib.pic[i]>thresh]=255
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        largest=0
#        print(i)
        for c, contour in enumerate(contours):
            Area = cv.contourArea(contour)
#            print(c,Area)
            if Area>=largest:
                largest=Area
#                circle=cv.arcLength(contour, True)
                contour_r=contour.reshape(contour.size//2,2)
                maxdist=spatial.distance_matrix(contour_r, contour_r).max()
#        liblist.append([largest,circle])
        liblist.append([largest,maxdist])
#        print(largest,circle*circle/(largest*4*3.1415926))
#        maxcd=circle*circle/(largest*4*3.1415926)
    liblist=np.asarray(liblist)
    thresh_contour = liblist[spatial.ConvexHull(liblist).vertices]
    return thresh_contour
def Extract_Protein(
    picdir,thresh_contour,
    outdir='test',
    cviter=3,
    picave=3,
    diiter=3,
    thresh_adt=0.35,
    thresh_pick=0.1,
    kernalsize_adt=(25,25),
    kernalsize_filter=(5,5),
    avekernal=[0.25,0.5,0.25],
    listthresh=30
    ):
    os.mkdir(outdir+'/binary')
    os.mkdir(outdir+'/ori')
#################################################################################
    extended_contour=np.zeros((thresh_contour.shape[0]+1,thresh_contour.shape[1]))
    extended_contour[:-1,:]=thresh_contour
    avekernal=np.asarray(avekernal)
    avecenter=avekernal.argmax()
    dirlist=os.listdir(picdir)
    pinserie=[]
    contourlist=[]
    print(thresh_contour)
    hullarea=spatial.ConvexHull(thresh_contour).volume
    for i,p in enumerate(dirlist):
        pinframe=[i+avecenter,dirlist[i+avecenter]]
        if i>=(len(dirlist)-picave):
            break
        ave=0
        ori=cv.imread(picdir+'/'+dirlist[i+avecenter],flags=cv.CV_8U)
        for j in range(picave):
            ave=ave+cv.imread(picdir+'/'+dirlist[i+j],flags=cv.CV_8U)*avekernal[j]
        burr=cv.GaussianBlur(ave.astype('uint8'), kernalsize_filter, 0)
        bi_gray= (255-AdaptiveThresh(burr,kernalsize_adt,thresh_adt)).astype('uint8')
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        cv.imwrite(outdir+'/binary/'+dirlist[i+avecenter],bi_gray)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        outpic=ave.copy()
#        tempmask=np.zeros(ave.shape,dtype='uint8')
        pcounts=0
        for c, contour in enumerate(contours):
            # print(i)
#            print(contour)
            Area = cv.contourArea(contour)
            if Area==0:
                continue
#            circle=cv.arcLength(contour, True)
            contour_r=contour.reshape(contour.size//2,2)
            maxdist=spatial.distance_matrix(contour_r, contour_r).max()
            extended_contour[-1]=np.asarray([Area,maxdist])
            para_pick=spatial.ConvexHull(extended_contour).volume-hullarea
            if Area>=50:
                contourlist.append([p,c,Area,maxdist,contour.mean(axis=0)[0][0],contour.mean(axis=0)[0][1],para_pick])
            if para_pick<thresh_pick:
                cv.drawContours(ori,contours,c,255,1)
                pinframe.append([contour.mean(axis=0)[0],contour])
                pcounts+=1
        pinserie.append(pinframe)
#        print('found ',pcounts,' in pic ',dirlist[i+avecenter])    
#        tempmask=cv.dilate(tempmask,kernel=kernel,iterations=3)
#        masked=(tempmask/255*(255-cv.imread(picdir+'/'+dirlist[i+avecenter],flags=cv.CV_8U))).astype('uint8')
        cv.imwrite(outdir+'/ori/'+dirlist[i+avecenter],ori)
    pd.DataFrame(contourlist,columns=['name','id','area','circle','x','y','para_dist']).to_csv(outdir+'/contour.csv')
    return pinserie
def Particle_clustering(pinserie,minobserve,maxmissing=15,maxdrift=7.0):
    plist=[]
    protein=[]
#pinserie saved as [[frame,filename,[center1,contour1],[center2,contour2]...]...]
#clustered protein saved as [[frame1,filename1,center1,contour1], [frame2,filename2,center2,contour2], [frame3,filename3,center3,contour3]...] 
    while len(pinserie)>0:
#        print(len(pinserie),pinserie[0],len(pinserie[0]))
        while len(pinserie[0])==2:
            pinserie.pop(0)
            if len(pinserie)==0:
                if len(protein)>=minobserve:
                    print('protein ',len(plist),'detected in ',len(protein),' frames')
                    plist.append(protein)
#                print('all protein detected')
                return plist
        protein=[[pinserie[0][0],pinserie[0][1],pinserie[0][2][0],pinserie[0][2][1]]]
        print('protein',len(plist),' detected at frame ',protein[-1][0],' centered at ',protein[-1][2])
        pinserie[0].pop(2)
        missedframe=1
        for pl in pinserie[1:]:
#            print(pl[0],len(pl))
            for pi in range(2,len(pl)):
#protein drift less than sqrt(maxdrift*missingframes) been considered as series
                if ((protein[-1][2]-pl[pi][0])**2).sum() <missedframe*maxdrift*maxdrift:
#                    print('find protein',len(plist),' at frame ',pl[0],'centered at',pl[pi][0],'after ',missedframe,' frames')
                    protein.append([pl[0],pl[1],pl[pi][0],pl[pi][1]])
                    pl.pop(pi)
                    missedframe=0
                    break
            missedframe+=1
            if missedframe>=maxmissing:
                break
        if len(protein)>=minobserve:
            print('protein ',len(plist),'detected in ',len(protein),' frames')
            plist.append(protein)
    return plist
def Gen_picked_pic(plist,dirin,outdir,
                   kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)),dilate_iter=3,picsize=[35,35],
                   debug=True):
    count=0
    dirlist=[]
    drifts=[]
    for i in plist:
        count+=1
        os.mkdir(outdir+'/protein_'+str(count))
        os.mkdir(outdir+'/ori_'+str(count))
#        print(os.system('mkdir '+tempdir+'_'+str(count)),'mkdir '+tempdir+'_'+str(count))
        dirlist.append(outdir+'/protein_'+str(count))
        drift=[]
        for c in i:
#            print(c[1],c[3])
            ori_pic=cv.imread(dirin+'/'+c[1],flags=cv.CV_8U)
            mask=np.zeros(ori_pic.shape,dtype='uint8')
            cv.drawContours(mask,(c[3],),0,255,cv.FILLED)
            mask=cv.dilate(mask,kernel=kernel,iterations=dilate_iter)
            masked=(mask/255*(255-ori_pic))
            masked[masked==0]=255-ori_pic.mean()
     #       masked=masked.astype('uint8')
            pos=[int(c[2][0]),int(c[2][1])]
            cutted=masked[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cutted_ori=ori_pic[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cv.drawContours(ori_pic,(c[3],),0,255,1)
            drift.append((max(0,pos[1]-picsize[1]),max(0,pos[0]-picsize[0])))
            cv.imwrite(outdir+'/ori_'+str(count)+'/'+c[1],cutted_ori)
            cv.imwrite(outdir+'/protein_'+str(count)+'/'+c[1],cutted)
        drifts.append(drift)
    return dirlist,drifts
def average_pic(picdir='2',ave=5,outdir='pic_average'):
    dirlist=os.listdir(picdir)
    os.system('mkdir '+outdir)
    frames=[]
    summary=0
    for i in range(ave):
        frames.append(mpimg.imread(picdir+'/'+dirlist[i]))
        summary=frames[i]+summary
        mpimg.imsave(outdir+'/0.png',summary/ave)
    count=0
    for i in range(ave,len(dirlist)):
        if count>=ave:
            count=0
        frames[count]=mpimg.imread(picdir+'/'+dirlist[i])
        mpimg.imsave(outdir+'/'+str(i-ave+1)+'.png',summary/ave)
    return

In [16]:
def makegif(fits,oridir,picdir,outdir,fit_shape,duration=0.3,level=50,cmap=cv.COLORMAP_JET):
#when fits==0 write gif for original pictures
    pics=os.listdir(picdir)
    os.mkdir(outdir)
    oriframes=[]
    simframes=[]
    for i in pics:
#        fit_shape=fits[i][0].shape
        oriframe=cv.imread(oridir+'/'+i)
        croped_pic=255-cv.imread(picdir+'/'+i)
        simframe=np.zeros(croped_pic.shape).astype('uint8')
        if oriframe.shape!=simframe.shape:
            print('hahaha')
            continue
        picshape=oriframe.shape
        if type(fits[i][0])==int or not i in fits:
            oriframes.append(oriframe)
            simframes.append(simframe)
            continue
        ys=fits[i][2]-fit_shape[1]
        xs=fits[i][1]-fit_shape[0]
        fm=fits[i][0].max()
        pic0=croped_pic[0,0,0]
        protein_mask=croped_pic[:,:,0][croped_pic[:,:,0]!=pic0]
        p_max=protein_mask.max()
        p_min=protein_mask.min()
        gray=(croped_pic-p_min).astype('float32')*255/(p_max-p_min)
        orifc=cv.applyColorMap(gray.astype('uint8'),cmap)
        for y in range(picshape[0]):
            for x in range(picshape[1]):
                if croped_pic[y,x,0]!=pic0:
                    oriframe[y,x]=orifc[y,x]
#        print(fits[i][0]>0.001)
#        op=np.zeros(pic.shape).astype('uint8')
#        op[:,:,0][pic[:,:,0]!=pic0]=255-protein_mask
#        op[:,:,1][pic[:,:,0]!=pic0]=(protein_mask)/3
#        op[:,:,2][pic[:,:,0]!=pic0]=(protein_mask)/1
#        oriframe[]
#        oriframe[fits[i][2]-len(fits[i][0]):fits[i][2],fits[i][1]-len(fits[i][0][0]):fits[i][1],:]=op
        oriframes.append(oriframe)
        print(i)
        template=(255-fits[i][0]*255/fm).astype('uint8')
        simfc=cv.applyColorMap(template,cmap)
        z=np.asarray([128,0,0]).astype('uint8')
        for x in range(fit_shape[0]):
            for y in range(fit_shape[1]):
                if template[y,x]<255:
                    simframe[fits[i][2]-len(fits[i][0])+y,fits[i][1]-len(fits[i][0][0])+x]=simfc[y,x]
#        print(fits[i][2]-len(fits[i][0]),fits[i][2],fits[i][1]-len(fits[i][0][0]),fits[i][1],simfc.shape)
#        simframe[fits[i][2]-len(fits[i][0]):fits[i][2],fits[i][1]-len(fits[i][0][0]):fits[i][1],:]=simfc
#        simframe[simframe[:,:,0]==0]=simfc[simframe[:,:,0]==0]
        print(oriframe.shape,simframe.shape,oriframe.dtype,simframe.dtype)
        simframes.append(simframe)
        cv.imwrite(outdir+'/ori_'+i+'.png',cv.cvtColor(oriframe,cv.COLOR_RGB2BGR))
        cv.imwrite(outdir+'/sim_'+i+'.png',cv.cvtColor(simframe,cv.COLOR_RGB2BGR))
    imageio.mimsave(outdir+'_ori.gif', oriframes, 'GIF', duration=duration)
    imageio.mimsave(outdir+'_fitted.gif', simframes, 'GIF', duration=duration)
    return

In [17]:
def cvmatch(lib,pic,drift,ori=0,r=10,method=cv.TM_CCOEFF_NORMED):
#def cvmatch(lib,pic,ori=0,r=10,method=cv.TM_CCORR_NORMED):
#def cvmatch(lib,pic,ori=0,r=10,method=cv.TM_CCORR):

#    tempic=(mpimg.imread(pic).mean()-mpimg.imread(pic)).astype('uint8')
#    inpic=cv.imread(pic,cv.CV_8U)[323:366,385:424].astype('float32')
#    inpic=cv.imread(pic,cv.CV_8U)[270:372,425:517].astype('float32')
    tempic=cv.imread(pic,cv.CV_8U).astype('float32')
#    tempic=extract_protein(inpic)
    bestfit=[0,0,0,0,0,0,0]
    cvlib=lib
    scores=[]
    drifts=[]
#    for p in lib:
#        cvlib[p]=(lib[p].astype('float32'))
    for p in cvlib.index:
        i=cvlib.pic[p]
        cmatch=cv.matchTemplate(tempic,i,method)
        m=cmatch.argmax()
        l=len(cmatch[0])
# normalized TMCCORR
#        score=2*cmatch.max()/((i*i).sum()+(tempic[int(m//l):int(m//l)+i.shape[0],m%l:m%l+i.shape[1]]*tempic[int(m//l):int(m//l)+i.shape[0],m%l:m%l+i.shape[1]]).sum())
        score=cmatch.max()
        scores.append(score)
        drifts.append([m%l+i.shape[1],int(m//l)+i.shape[0]])
    return drifts,np.asarray(scores)
def extract_best(scores,libshape,ntops=5):
    score_t=scores.reshape(libshape[1],libshape[0])
    output=[]
    for pid,score_l in enumerate(score_t):
        top_index=np.argpartition(score_l, -ntops)[-ntops:]
        for i in top_index:
            output.append([pid,i,score_l[i]])
    return(np.asarray(output))
def autosearch(picdir,template,
               outdir='fitted',debug=True,
               quatfile='lib/c48u27.quat',resolution=3.7,rf=10,pick=[],
               libmol='lib/mol.csv',iscg=0,picsize=0,thresh_pick=0.01,
               minobserve=100,thresh_adt=0.35,ntops=1):
    dirlist=os.listdir(picdir)
    os.mkdir(outdir)
    if type(template)==str:
        lib,size=genlib(template,quatfile,resolution=resolution,iscg=iscg,libmol=libmol,size=picsize)
        picsize=[size,size]
        print('lib generated')
        thresh_contour=gen_Morph_parameter(lib)
    elif type(template)==pd.DataFrame:
        lib=template
        print('use input lib')
        picsize=lib.pic[0].shape
        thresh_contour=gen_Morph_parameter(lib)
    else:
        raise ValueError('input lib must be a folder contains pdb files or a libary generated by genlib')
    if len(pick)==3:
#            thresh_contour=np.asarray([[pick[0]*pick[1]*np.pi/(4*resolution),max(pick[0],pick[1])],
#                                       [pick[2]*pick[1]*np.pi/(4*resolution),max(pick[2],pick[1])],
#                                       [pick[0]*pick[2]*np.pi/(4*resolution),max(pick[0],pick[2])],])/resolution
            thresh_contour=np.asarray([[pick[0]*pick[1]/(resolution),np.sqrt(pick[0]*pick[0]+pick[1]*pick[1])],
                                       [pick[2]*pick[1]/(resolution),np.sqrt(pick[2]*pick[2]+pick[1]*pick[1])],
                                       [pick[0]*pick[2]/(resolution),np.sqrt(pick[0]*pick[0]+pick[2]*pick[2])],])/resolution
            print('pick particles with length: ',template)
    print('pick range generated')
    pinserie=Extract_Protein(picdir,thresh_contour,outdir=outdir,thresh_adt=thresh_adt,thresh_pick=thresh_pick)
    print('contours found')    
    plist=Particle_clustering(pinserie,minobserve,maxmissing=30,maxdrift=7)
    print('particle clustered')
    picked_dirlist,drifts=Gen_picked_pic(plist,picdir,outdir=outdir,
                                         kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3)),dilate_iter=3,picsize=picsize)
    print('picked picture generated')
    summary=pd.DataFrame({'pic':[],'protein_id':[],'pdb':[],'score':[],'qr':[],'qi':[],'qj':[],'qk':[],'dx':[],'dy':[]},index=[])
    pdblist=lib['pdb'].unique()
    libshape=[int(len(lib)/len(pdblist)),len(pdblist)]
    qlist=lib['q'][0:libshape[0]]
    for pc,protein in enumerate(picked_dirlist):
        fits={}
        dirlist_pro=os.listdir(protein)
        list_score=np.zeros([len(dirlist_pro),len(lib)])
        for c,i in enumerate(dirlist_pro):
            pic_summary=pd.DataFrame({'pic':[],'protein_id':[],'pdb':[],'score':[],'qr':[],'qi':[],'qj':[],'qk':[],'dx':[],'dy':[]},index=[])
            drift,list_score[c]=cvmatch(lib,protein+'/'+i,drifts[pc][c])
            outdata=extract_best(list_score[c],libshape,ntops)
            for out in outdata:
#               print(qlist[out[1]])
#                for n in range(len(out)):
#                print(out)
                pic_summary=pic_summary.append(pd.DataFrame({'pic':c,'protein_id':pc,'pdb':pdblist[int(out[0])],'score':out[2],
                                                     'qr':qlist[out[1]][0].item(),'qi':qlist[out[1]][1].item(),'qj':qlist[out[1]][2].item(),'qk':qlist[out[1]][3].item(),
                                                     'dx':drift[int(out[0]*libshape[1]+out[1])][1],'dy':drift[int(out[0]*libshape[1]+out[1])][0]},index=[0]),ignore_index=True)
#                print(outdata)
            best_index=pic_summary.score.argmax()
#            print(pic_summary.dx[best_index])
            fit=[0,pic_summary.dx[best_index],pic_summary.dy[best_index],pic_summary.score[best_index],
                 pic_summary.pdb[best_index]]
            summary=summary.append(pic_summary,ignore_index=True)
            fits[i]=fit
            print('protein '+protein+' in picture',i,'is fitted as',fit[4],'fitted score:',fit[3])
        makegif(fits,outdir+'/ori_'+str(pc+1),outdir+'/protein_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'pics',picsize)
        print('gif for protein '+str(pc+1)+' saved as '+outdir+'/protein_'+str(pc+1)+'movie.gif')
        table_score=pd.DataFrame(list_score)
        table_score.index=dirlist_pro
        table_score.to_csv(outdir+'/protein_'+str(pc+1)+'.csv')
#        makegif(0,outdir+'/ori_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'ori.gif')
    summary.to_csv(outdir+'/summary.csv')
    return summary
def drawcmatch(pdblib='GAPRPDB',lib='',picdir='2',outname='sim.gif',ang=5,libgrid='std60.csv',
         resolution=3.7,rf=10,oriout='',libmol='mol.csv',iscg=0,picaver=0):
    if picaver:
        os.system('mkdir TEMPPIC')
        average_pic(picdir,ave=picaver,outdir='TEMPPIC')
        picdir='TEMPPIC'
    if lib:
        print('use input lib')
    else:
        lib=genlib(pdblib,libgrid,ang,resolution=resolution,iscg=iscg,libmol=libmol)
        print('lib generated')
    cmatch=[]
    dirlist=os.listdir(picdir)
    for i in dirlist:
        cmatch.append(cvmatch(lib,picdir+'/'+i))
    print('gif saved as ',outname)
    return cmatch

In [18]:
libecori,size=genlib(pdblib='PDBfiles/state228',quatfile='lib/c48u309.quat',resolution=0.5,iscg=1,libmol='lib/sirah2-liu.csv')

  0%|                                                                                         | 0/7416 [00:00<?, ?it/s]<ipython-input-14-d39382a9d121>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  atom_x=torch.tensor((pos[:,0]+drift[0])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
<ipython-input-14-d39382a9d121>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  atom_y=torch.tensor((pos[:,1]+drift[1])/resolution,dtype=torch.int32)#.type(torch.ShortTensor)
100%|█████████████████████████████████████████████████████████████████████████████| 7416/7416 [00:14<00:00, 517.86it/s]


In [19]:
fits_ecori=autosearch(picdir='simulated_data',template=libecori,outdir='test1',rf=10,minobserve=1,thresh_adt=0.15,debug=True,thresh_pick=10000)

use input lib


UnboundLocalError: local variable 'maxdist' referenced before assignment